In [1]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install -U peft
!pip install huggingface_hub

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN_READ = user_secrets.get_secret("HF_TOKEN")

In [3]:
!huggingface-cli login --token $HF_TOKEN_READ

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
new_model_id = "APaul1/Llama-3-8B-sft-lora-ultrachat"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from transformers import BitsAndBytesConfig

2024-04-26 22:50:37.809075: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 22:50:37.809142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 22:50:37.815333: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
device_map = "auto"

In [7]:
# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)

In [15]:
# base_model_reload = AutoModelForCausalLM.from_pretrained(
#         base_model_id,
#         return_dict=True,
#         low_cpu_mem_usage=True,
#         torch_dtype=torch.float16,
#         device_map="auto",
#         trust_remote_code=True,
# )
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                             quantization_config=quantization_config, 
                                             device_map=device_map)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
model = PeftModel.from_pretrained(base_model, new_model_id)

adapter_model.safetensors:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [18]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [20]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [21]:
outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [22]:
response = outputs[0][input_ids.shape[-1]:]

In [23]:
print(tokenizer.decode(response, skip_special_tokens=True))

Arrrr, shiver me timbers! Ye be askin' a mighty strange question, matey! Helicopters be flyin' machines, not grub fer ye stomach! Ye can't eat a helicopter, matey, no matter how hungry ye be! It'd be as silly as tryin' to sail a ship on land or swabbin' the decks with a spoon! So, the answer be zero, matey! No human can eat a helicopter in one sittin' or any sittin' fer that matter! Now, if ye be lookin' fer a tasty meal, I can recommend


In [24]:
messages = [
    {"role": "system", "content": "Alan Watts's colorful journey explored the depths of philosophy and religion, weaving together Eastern wisdom and Western insights."},
    {"role": "user", "content": "What does the self actually amount to?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [28]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [29]:
response = outputs[0][input_ids.shape[-1]:]

In [30]:
print(tokenizer.decode(response, skip_special_tokens=True))

A question that has puzzled philosophers and spiritual seekers for centuries! According to Alan Watts, a renowned philosopher and writer, the concept of the self is a myth, a mere illusion. He argued that the self is not a fixed, essential entity, but rather a dynamic, ever-changing process.

Watts believed that the self is a social construct, created by our interactions with others and our environment. He saw the self as a kind of "social fiction," a narrative we tell ourselves to make sense of our experiences and to distinguish ourselves from others.

In his book "The Book on the Taboo Against Knowing Who You Are," Watts wrote, "The self is not something ready-made, it is something that is constantly being created and recreated in the process of living."

Watts's perspective is rooted in Eastern philosophy, particularly Buddhism and Taoism. He believed that the self is an illusion, a mere appearance, and that true reality lies beyond the boundaries of the ego. He argued that our sens